In [ ]:
import numpy as np
from itertools import product
import matplotlib as mpl
from matplotlib import rc
import matplotlib.pyplot as plt
from plotutils_1d import *
# from scipy.stats import linregress
plt.style.use('default')
# plt.style.use('dark_background')
%config InlineBackend.figure_format = 'svg'

rc('font', **{'family': 'serif'})
rc('text', usetex=False)
mpl.rcParams['font.size'] = 12
np.set_printoptions(precision=4, suppress=True)
tvbm = [0, 0.25, 1, 1.25, 2, 2.25, 3, 3.25]

## Fit Relation between Parameter and Radius

The compactification radius of compactified boson CFT is

$$
\Delta_{m,n} = \frac{m^2 R^2}{4} + \frac{n^2}{R^2}
\Rightarrow
R^2 = \frac{2 \left(\Delta + \sqrt{\Delta ^2-m^2 n^2}\right)}{m^2}
$$

For the lowest level $\Delta = 0.25$, we should set $m = 1, n = 0$, i.e.

$$
\Delta_{1,0} = \frac{R^2}{4}
\Rightarrow
R = 2 \sqrt{\Delta_{1,0}}
$$

Then we can fit data to find the relation between $R$ and the adjustable parameter ($g, \mu$, etc)

<!-- Alternatively, we may also use the 2nd lowest level $m = 0, n = 1$, i.e.

$$
\Delta_{0,1} = \frac{1}{R^2} 
\Rightarrow
R = \frac{1}{\sqrt{\Delta_{0,1}}}
$$ -->

In [ ]:
def scdim_exact(m, n, r2):
    """Exact scaling dimension of compactified boson CFT"""
    return m**2 * r2 / 4 + n**2 / r2

## tV model

In [ ]:
def gather_dim2(datadir, gname, mattype, pltstep, fixvar=dict(), grange=None):
    sec1 = gather_dim(datadir, gname, mattype+'-a0', pltstep, fixvar, grange)
    sec2 = gather_dim(datadir, gname, mattype+'-p1', pltstep, fixvar, grange)
    return combine_dim(sec1, sec2)

In [ ]:
# datadir = "tV-bm-noflt/"
datadir = "tV-bm2/"
gname = 'g'
pltstep = 14

In [ ]:
# fixvar = {'beta': 1.00}
fixvar = {}
dimh = gather_dim2(datadir, gname, 'd4h', pltstep, fixvar, grange=(0., 0.9))
dimv = gather_dim2(datadir, gname, 'd4v', pltstep, fixvar, grange=(0., 0.9))

In [ ]:
dimv, dimh, vlist = correct_scdim(dimv, dimh, return_v=True)
vlist

In [ ]:
# observe the relation between R and parameter
fig, ax = plt.subplots(1, 1, figsize=(2, 2))
ax.set_xlabel(r'$g$', fontsize=14)
ax.set_ylabel(r"$R^2$", fontsize=14)
ax.tick_params(axis='both', direction='in', labelsize=14)
func = ax.scatter
kwargs = dict(s=4)

xdata = dimh[:,0]
ydata = dimh[:,2] * 4
func(xdata, ydata, **kwargs)

In [ ]:
# use polynomial fit
deg = 1
coeff = np.polyfit(xdata, ydata, 1)
coeff

In [ ]:
def r2_fit(x, coeff):
    y = 0
    for i in range(deg + 1):
        y += coeff[i] * x**(deg-i)
    return y

In [ ]:
ax.plot(xdata, [r2_fit(x, coeff) for x in xdata], label='fit')
# compare with exact result
# R^2 = 2 * (1-g)
ax.plot(xdata, [2 * (1 - x) for x in xdata], label='exact')
ax.legend()
fig

## Plot all sectors and compare with CFT prediction

In [ ]:
# load data
dimh = gather_dim2(datadir, gname, 'd4h', pltstep, fixvar, grange=(0.0, 0.9))
dimv = gather_dim2(datadir, gname, 'd4v', pltstep, fixvar, grange=(0.0, 0.9))
# dimv, dimh = correct_scdim(dimv, dimh)

In [ ]:
def r2_exact(g):
    return 2 * (1-g)

In [ ]:
# create canvas
fig, ax = plt.subplots(1, 5, figsize=(10, 5), sharey=True)
#     fig.patch.set_facecolor('#000000')
# adjust spacing
plt.subplots_adjust(left=0.06, bottom=0.10, right=0.97, top=0.9, 
                    wspace=0.1, hspace=0.3)
for axis in ax:
    axis.set_xlabel(r"$g$")
    axis.tick_params(axis='both', direction='in')
    axis.set_xlim(-0.05, 1.05)
    axis.grid(which='both', axis='both')
ax[0].set_ylabel(r"$\Delta$")
ax[0].set_ylim(bottom=-0.1, top=2.8)

# plot reference lines
# for i in range(len(tvbm)):
#     for axis in ax:
#         axis.axhline(y=tvbm[i], color='green', linestyle=':')
#         axis.axvline(x=0.5, color='red', linestyle=':')
        
# plot each sector
nev = 80
bclist = ['a0', 'a1', 'p0', 'p1']
mattype = 'd4h'
grange = (0.0, 0.9)
scale = 1.0
titles = ['APBC\nEven parity', 'APBC\nOdd parity', 
         'PBC\nEven parity', 'PBC\nOdd parity']
for i, (bc, title) in enumerate(zip(bclist, titles)):
    ax[i].set_title(title)
    ax[i].set_xlabel(r'$g$')
    ax[i].tick_params(axis='both', direction='in')
    scdim = gather_dim(datadir, gname, '-'.join([mattype, bc]), pltstep, fixvar, grange)
    scdim = scdim[:,0:(nev+1)]
    xdata = scdim[:,0]
    for j in range(1, scdim.shape[1]):
        ydata = scdim[:,j] * scale
        ax[i].scatter(xdata, ydata, s=3, color='blue')

# plot CFT prediction
ax[4].set_title('Compare\nwith CFT')
xdata = dimh[:, 0]
# already obtained R^2 = a x + b
for m, n, p in product(range(4), range(4), range(4)):
    # xdata = np.sort(xdata)
    ydata = [scdim_exact(m, n, r2_exact(x))*scale + p for x in xdata]
    ax[4].plot(xdata, ydata, color='red', linewidth=0.5)
ax[4].plot(xdata, ydata, color='red', linewidth=0.5, label='CFT')
    
# plot full scaling dimension
for i in range(1, dimh.shape[1]):
    ydata = dimh[:, i]*scale
    ax[4].scatter(xdata, ydata, color='blue', s=3)
ax[4].scatter(xdata, ydata, color='blue', s=3, label='Loop-TNR')
ax[4].legend(fontsize=9)

In [ ]:
# fig.savefig('radius.pdf')